In [1]:
from src.neo4jdataloader import DataLoader
from src.neo4jcrud import UrbsNeo4JDatabase

In [2]:
NEO4J_URI = 'bolt://172.17.0.2:7687'
NEO4J_USER = 'neo4j'
NEO4J_PASSWORD = 'neo4j2018'

conn = UrbsNeo4JDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
#conn.delete_all()

In [3]:
dataloader = DataLoader()

In [ ]:
trechositinerarios_source_path = '/home/altieris/datascience/data/curitibaurbs/processed/trechositinerarios/'
trechositinerarios = dataloader.load_data(trechositinerarios_source_path)
dataloader.create_empresas_onibus(trechositinerarios, conn)
dataloader.create_categorias_onibus(trechositinerarios, conn)

In [11]:
pontos_linha_source_path = '/home/altieris/datascience/data/curitibaurbs/processed/pontoslinha/'
pontos_linha = dataloader.load_data(pontos_linha_source_path)
dataloader.create_bus_stops(pontos_linha,conn)

In [12]:
dataloader.create_routes(conn)

In [5]:
veiculos = dataloader.load_data('/home/altieris/datascience/data/curitibaurbs/processed/veiculos/')

dataloader.create_vehicles(veiculos,conn)

In [5]:
veiculos = dataloader.load_data('/home/altieris/datascience/data/curitibaurbs/processed/veiculos/')
dataloader.create_positions(veiculos,conn)

AttributeError: 'DataFrame' object has no attribute 'iterrows'

In [ ]:
conn.close()